In [1]:
from importlib import reload
import nbimporter
from collections import Counter
import ReadBedFiles
from ReadBedFiles import readJsonFile, readFiles2Vector, writeJsonFile, convertMat2document, readJsonFile
import glob
import pandas as pd
import numpy as np
import time
import copy
import gc
from gensim.models import Word2Vec
from sklearn.decomposition import PCA

Importing Jupyter notebook from ReadBedFiles.ipynb
Importing Jupyter notebook from AIListFunc.ipynb


In [2]:
import numpy as np
from gensim.models import Word2Vec
import json

In [3]:
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import matplotlib.font_manager as font_manager

In [4]:
def initialization(clas_type, label, path_train, path_test, path_universe, path_mat, meta_data, sample_of_interest, path_w2v_model):
    path_train = path_train.format(clas_type)
    path_test = path_test.format(clas_type)
    path_universe = path_universe.format(clas_type)
    path_mat = path_mat
    meta_data = pd.read_csv(meta_data.format(clas_type))
    meta_data = meta_data.loc[meta_data[label].isin(sample_of_interest)][['Experiment_ID', label]]
    model = Word2Vec.load(path_w2v_model.format(clas_type))
    
    return path_train, path_test, path_universe, path_mat, meta_data, model

In [5]:
def embedding_avg(model, document):
    listOfWVs= []
    for word in document.split(' '):
        if word in model.wv.vocab:
            listOfWVs.append(model[word])
            
    if(len(listOfWVs) == 0):
        return np.zeros([100])
    return np.mean(listOfWVs, axis=0)

def document_embedding_avg(document_Embedding, model):
    document_Embedding_avg = {}
    for file, doc  in document_Embedding.items():
        document_Embedding_avg[file] = embedding_avg(model, doc)
    return document_Embedding_avg

def create_data4T_SNE(tdMatrix, labels, label, threshld, clm):
    tdMatrix =  {k.lower(): v for k, v in tdMatrix.items()}
#     print(len((set(labels.Experiment_ID))))
    labels.Experiment_ID = list(labels.Experiment_ID.str.lower())
#     print(labels.Experiment_ID[0:10])
    commonFiles = list(set(tdMatrix.keys())  & set(labels.Experiment_ID)) 
#     print(len(commonFiles))
    y = []
    for name in commonFiles:
        y.append(labels[labels[clm] == name][label].tolist()[0])
    list_of_frequent= y #removeElements(y, threshld)
#     print(len(list_of_frequent))
    data_X = []
    y = []
    for name in commonFiles:
        y_1 = labels[labels[clm] == name][label].tolist()[0]
        if(y_1 in list_of_frequent):
            data_X.append(tdMatrix[name])
            y.append(y_1)
    
    return np.array(data_X), y

# Read data

In [36]:
i = 1

In [37]:
clas_types = ['antibody', 'cell', 'tissue']
labels = ['antibody', 'cell line', 'tissue']
sample_of_interest = [['h3k27ac', 'h3k4me3', 'h3k27me3', 'h3k4me1', 'h3k36me3', 'h3k9me3', 'h3k4me2'],
                     ['k562', 'mcf7', 'hek293', 'a549', 'hepg2', 'hct116','lovo', 'gm12878', 'lncap','hela'],
                     ['liver', 'peripheral blood', 'primary prostate cancer', 'blood', 'breast','bone marrow', 'kidney']
                     ]
PCA_flg = False
label = labels[i]
clas_type = clas_types[i]

In [38]:
results = pd.DataFrame(columns = ['dataset','representation',  'pca', 'f1'])

In [39]:
path = '/project/shefflab/www/papers/region_embedding/'
path_train = path + 'datasets/{}dataset/train/*'
path_test = path + 'datasets/{}dataset/test/*'
path_universe = './representations/{}/'
path_mat = path + 'datasets/term_doc_mat/'
meta_data = './meta_data/meta_data_{}.csv'
path_w2v_model = path + 'word2vecmodels/word2vec_{}.model'

In [40]:
path_train, path_test, path_universe, path_mat, meta_data, model = initialization(clas_type, label, path_train, path_test, path_universe, path_mat, meta_data, sample_of_interest[i], path_w2v_model)


In [43]:
path_univ

'./representations/cell_universe.txt'

In [ ]:
%%time
path_univ = './representations/cell_universe.txt'

pca = PCA(n_components = 100)

print(path_univ)
train_files , segmentation_df_train = readFiles2Vector(path_train, path_univ, numberofCores = 4, numOfFiles= 100, PATH = path_train)
print(len(train_files))

test_files, segmentation_df_test = readFiles2Vector(path_test, path_univ, numberofCores = 4, numOfFiles= 100, PATH = path_test)
print(len(test_files))

document_Embedding_train = convertMat2document(train_files, segmentation_df_train)
document_Embedding_avg_train = document_embedding_avg(document_Embedding_train, model)

document_Embedding_test = convertMat2document(test_files, segmentation_df_test)
document_Embedding_avg_test = document_embedding_avg(document_Embedding_test, model)

X_train, y_train = create_data4T_SNE(document_Embedding_avg_train, (meta_data.loc[(meta_data[label] !='none') & (meta_data[label] !='nan') & (meta_data[label].notna())]), label, 0, 'Experiment_ID')
X_test, y_test = create_data4T_SNE(document_Embedding_avg_test, (meta_data.loc[(meta_data[label] !='none') & (meta_data[label] !='nan') & (meta_data[label].notna())]), label, 0, 'Experiment_ID')

print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

clf = svm.SVC(kernel = 'linear')
start_time = time.time()
clf.fit(X_train, y_train)

f1 = f1_score((y_test), clf.predict(X_test), average = 'micro')
print(f1_score((y_test), clf.predict(X_test), average = 'micro'))

results = pd.concat([results, pd.DataFrame([[clas_type, 'Region-set2vec', PCA_flg, f1]], columns= ['dataset', 'representation',  'pca', 'f1'])], ignore_index=True)

results.to_csv('./results/F1_class{}_pca{}.csv'.format(clas_type, PCA_flg), index = False)

./representations/cell_universe.txt
Reading universe file: Done 2021-04-08 14:57:00.060729
7003
